# OpenAI Agent with LlamaIndex

## Install Dependencies

In [1]:
!pip install uv
!uv pip install -qU xpander-sdk llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 15.7 MB/s eta 0:00:00a 0:00:01


## Setup API Keys


In [2]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

## Import libraries and setup LlamaIndex

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4o")

# Set up Phoenix

In [4]:
import phoenix as px
session = px.launch_app()

/Users/dudu/Developer/xpander-lab/projects/xpander-agents-hub/agents/llama-index/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
from xpander_sdk import XpanderClient, LLMProvider
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
# create execution
xpander_agent.add_task("Get only the longest readable tag")
print(xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI))
print("Input:",xpander_agent.execution.input_message.content)
print("General:",xpander_agent.instructions._delegates[0].general)
print("Goal:",xpander_agent.instructions._delegates[0].goal)
print("Role:",xpander_agent.instructions._delegates[0].role)
print("-"*100)
print("Messages object:")
for message in xpander_agent.memory.messages:
    print("-"*100)
    print("Role:",message.role)
    print("Tool Call ID:",message.tool_call_id)
    print("Tool Calls:",message.tool_calls)
    print("Content:",message.content)
print("-"*100)

b'\x1b\x1b!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software has not been tested with node v23.4.0.                                                                !!\x1b\x1b\n'
b'\x1b\x1b!!  Should you encounter odd runtime issues, please try using one of the supported release before filing a bug report.  !!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software is currently running on node v23.4.0.                                                                 !!\x1b\x1b\n'
b'\x1b\x1b!!  As of the current release of this software, supported node releases are:                                            !!\x1b

[{'type': 'function', 'function': {'name': 'HackerNewsStoriesRetrieveNewStoryIdsList', 'description': 'Obtains a list of IDs for the most recent stories posted on Hacker News. This operation provides access to the latest content across all categories. Use it to find story IDs for further details retrieval using RetrieveItemDetailsById. Ideal for creating real-time feeds, monitoring breaking news, or analyzing the influx of new content on the platform.', 'parameters': {}}}, {'type': 'function', 'function': {'name': 'LinkedInCompanyManagementGetCompanyDetailsByUsername', 'description': "Retrieves comprehensive information about a company using its LinkedIn username. This operation provides valuable insights into company profiles, including size, industry, and location. If the username isn't provided, run GetCompanyInfoByDomain if you have the company's domain name. Use this to gather detailed company information for business intelligence, partnership opportunities, or competitive analysi

In [6]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [7]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [8]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True)
    uber_index = VectorStoreIndex.from_documents(uber_docs, swow_progress=True)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

Now create the query engines.

In [9]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3, llm=llm)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [10]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [11]:
llama_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [12]:
response = llama_agent.chat("Who had more profit in 2021, Lyft or Uber?")
print(str(response))

> Running step 75ad006c-3f8c-4fbb-94e2-85a260374b2e. Step input: Who had more profit in 2021, Lyft or Uber?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's profit in 2021?"}
Observation: Lyft did not report a profit in 2021; instead, it reported a net loss of $1.0 billion for the year.
> Running step ac7a1d7c-d122-4f84-999e-7d963ed4d9f5. Step input: None
Thought: I have the information about Lyft's financial performance in 2021. Now, I need to find out Uber's profit for the same year to compare.
Action: uber_10k
Action Input: {'input': "What was Uber's profit in 2021?"}
Observation: Uber did not make a profit in 2021. The company reported a net loss attributable to Uber Technologies, Inc. of $496 million for that year.
> Running step 56e54268-1c76-40a1-a8d4-0c9df44ba9d1. Step input: None
Thought: I have the necessary information about both Lyft's and Uber's financial pe

In [2]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")

# get the agent
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
openai_client = OpenAI(api_key=OPENAI_API_KEY)
# create execution
xpander_agent.add_task("Who had more profit in 2021, Lyft or Uber?")
while not xpander_agent.is_finished():
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice="auto",
                temperature=0.0
        ) 
    print("model response", response.model_dump())
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    for tool_call in tool_calls:
        print("tool name: ", tool_call.name)
        print("tool call generated payload: ", tool_call.payload)
    tool_responses = xpander_agent.run_tools(tool_calls=tool_calls)
    for tool_response in tool_responses:
        print("tool name: ", tool_response.function_name)
        print("tool status code: ", tool_response.status_code)
        print("tool result: ", tool_response.result)
    if response.choices[0].finish_reason == "stop":
        break
print(xpander_agent.retrieve_execution_result().result)

----------------------------------------------------------------------------------------------------
model response {'id': 'chatcmpl-AvbSp3OWCegJe0dE3ZhYjWO8CD3Bo', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'In 2021, Uber reported a net income of $0.9 billion, while Lyft reported a net loss of $1.0 billion. Therefore, Uber had more profit in 2021 compared to Lyft.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}}], 'created': 1738292071, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_50cad350e4', 'usage': {'completion_tokens': 46, 'prompt_tokens': 2213, 'total_tokens': 2259, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2048}}}

